In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from statsmodels.tsa.arima.model import ARIMA


In [2]:
new_df = pd.read_csv('Archivos/NewDataAll.csv', low_memory=False)
causas_vih = pd.read_csv('Archivos/causas_vih.csv', low_memory=False)
enfermedades = pd.read_excel('Archivos/CIE10.xlsx')



In [3]:
enfermedades2 = enfermedades.copy()
enfermedades2.loc[enfermedades2['CAUSA'].str.len() == 3, 'CAUSA'] = enfermedades2.loc[enfermedades2['CAUSA'].str.len() == 3, 'CAUSA'] + "X"
enfermedades2

new_df2 = pd.merge(new_df, enfermedades2, left_on='caudef', right_on='CAUSA', how='left')

new_df2 = new_df2.dropna()
causas_infecciones = new_df2.loc[new_df2["DESCRIPCION"].str.contains("Infecciones|infecciones|infeccion|Infección|Infeccion|infección")]

causas_vih_index = new_df2.loc[new_df2["DESCRIPCION"].str.contains("VIH|vih|SIDA")].index


In [4]:
new_df2['VIH'] = 0
new_df2.loc[causas_vih_index, 'VIH'] = 1


Datos sobre que tanto subieron o bajaron las muertes por vih

In [5]:
tf_cvih = pd.crosstab(causas_vih['añoocu'], columns='count')
v09 = tf_cvih['count'][2009]
v20 = tf_cvih['count'][2020]

d1 =(abs(v09-v20)/v09)*100
d1

63.53503184713376

Datos sobre que tanto subieron las muertes desde 2009 hasta 2020

In [6]:
tf_ndf = pd.crosstab(new_df['añoocu'], columns='count')
d09 = tf_ndf['count'][2009]
d20 = tf_ndf['count'][2020]

dif =(abs(d09-d20)/d09)*100
dif

33.879537562581056

Cuanto ha bajado el porcentaje de muertes que representa el vih desde el 2009 hasta el 2020

In [7]:
por_vih09 = (v09/d09)*100 #0.876 porciento de las muertes en 2009 fueron causadas por vih
por_vih20 = (v20/d20)*100 #0.239 porciento de las muertes en 2020 fueron causadas por vih

dif_porvih = abs(por_vih09-por_vih20)*100/por_vih09
dif_porvih

72.76285172719471

# Logistic Regression

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

columns = ['depocu', 'sexo',
       'edadif', 'perdif', 'ecidif', 'dnadif',
       'nacdif', 'dredif', 'asist', 'ocur',
       'cerdef']

X = new_df2[columns]
y = new_df2['VIH']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression(class_weight='balanced')

model.fit(X_train, y_train)


c:\Users\Alien Ware\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced')

In [21]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

true_negative = confusion[0, 0]
false_positive = confusion[0, 1]
false_negative = confusion[1, 0]
true_positive = confusion[1, 1]

sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

# Imprimir los resultados
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)


print(classification_report(y_test, y_pred))



Accuracy: 0.7377606123625942
Sensitivity: 0.7768037761294673
Specificity: 0.7375571811033581
              precision    recall  f1-score   support

           0       1.00      0.74      0.85    284622
           1       0.02      0.78      0.03      1483

    accuracy                           0.74    286105
   macro avg       0.51      0.76      0.44    286105
weighted avg       0.99      0.74      0.84    286105



# SVM

In [16]:

# Get the indices of rows where caudef is 1 (HIV deaths)

new_df3 = new_df2.copy()
hiv_indices = new_df3[new_df3['VIH'] == 1].index

# Sample a portion of non-HIV death cases
non_hiv_sample = new_df3[new_df3['VIH'] == 0].sample(n=100000 - len(hiv_indices), random_state=42)

# Concatenate the HIV death cases and non-HIV death cases
sample = pd.concat([new_df3.loc[hiv_indices], non_hiv_sample])

# Shuffle the sample
sample = sample.sample(frac=1, random_state=42)

X = sample[columns]
y = sample['VIH']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [11]:
len(X_test)
len(X_train)

35000

In [17]:
from sklearn.svm import SVC



svm_model = SVC(kernel='rbf', C=1.0, gamma='auto', class_weight='balanced')
svm_model.fit(X_train, y_train)




print('done modelfit')
# Realizar predicciones en el conjunto de prueba
y_pred = svm_model.predict(X_test)
print('done ypred')
# Calcular la precisión (accuracy), sensibilidad (sensitivity) y especificidad (specificity)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

true_negative = confusion[0, 0]
false_positive = confusion[0, 1]
false_negative = confusion[1, 0]
true_positive = confusion[1, 1]

sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

# Imprimir los resultados
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)


print(classification_report(y_test, y_pred))

done modelfit
done ypred
Accuracy: 0.9411
Sensitivity: 0.12297297297297298
Specificity: 0.9835553997194951
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     28520
           1       0.28      0.12      0.17      1480

    accuracy                           0.94     30000
   macro avg       0.62      0.55      0.57     30000
weighted avg       0.92      0.94      0.93     30000



# Arima

In [ ]:
new_df4 = new_df2.copy()
new_df4['mesocu'] = new_df4.loc